In [4]:
import os
os.chdir('D:/duckdb/files/RNA2/')

In [5]:
import pyarrow.parquet as pq
import pandas as pd
import glob
import duckdb

con = duckdb.connect()
con.execute("PRAGMA threads=2") # enable automatic query parallelization
con.execute("PRAGMA enable_object_cache") # enable caching of parquet metadata

In [ ]:
%%time
print(con.execute("SELECT COUNT(distinct(CF)) CF  FROM 'anno*.parquet'").df())

In [ ]:
con.execute("select year(concessione) anno,  count(distinct(CF)) imprese, count(cf) aiuti, round(sum(nominale)/1000000, 1) nominale_M€ \
                  FROM 'anno*.parquet' group by year(concessione) order by anno desc;").df()

In [ ]:
con.execute("select replace(concedente,'AGENZIE DELLE ENTRATE','AGENZIA DELLE ENTRATE') concedente, count(distinct(CF)) imprese, \
count(cf) aiuti, round(sum(nominale)/1000000,1) nominale_M€ \
                  FROM 'anno*.parquet' where year(concessione)=2023 group by 1 order by 2 desc limit 20;").df()

In [7]:
import polars as pl

In [ ]:
%%time
df = pl.read_parquet('anno*.parquet',)

In [ ]:
df.shape

In [ ]:
df['concedente'] = df['concedente'].str.replace(r'"', '')

In [ ]:
df['concedente'] = df['concedente'].str.replace('AGENZIA ENTRATE','AGENZIA DELLE ENTRATE')

In [ ]:
df['concedente'] = df['concedente'].str.replace('AGENZIE DELLE ENTRATE','AGENZIA DELLE ENTRATE')

# filtrare il df

In [ ]:
# MCC
df = df[~(df.CAR.isin(["12627","15884","16617","16615","18730","526","24646","27308","23546"]))]

In [ ]:
df = df[~(df.CAR.isin(["19754","15959","20177","884","21788","17496","15880","18857","22941","18945"]))] # cancellata MCC

In [ ]:
# AGENZIA DELLE ENTRATE
df = df[~(df.CAR.isin(["19754","15959","20177","16100","17046","19795","17065","8152","16348","10373","16372","16088","16274"]))]

In [ ]:
# INPS, INAIL
df = df[~(df.CAR.isin(["21788","18857","18945","807","18852","6047","22076","10106","12728","6048","1522","22078","18982","18858"]))]

In [ ]:
# Formazione
df = df[~(df.CAR.isin(["884"]))]

In [ ]:
df.groupby(["concedente","CAR"], as_index=False).agg(aiuti = ("CF", "count"))\
    .sort_values(by="aiuti", ascending=False).head(40)

# ricerca

In [ ]:
df[df['CAR'] == '10106']

In [ ]:
df[df['CF'] == '00594040586']

# 3656 SME Initiative

- https://www.mimit.gov.it/it/incentivi/pon-iniziativa-pmi


In [ ]:
sme =(con.execute("SELECT year(concessione) anno, concedente, CF imprese,regione,nominale,aiuto FROM '*.parquet' where CAR = 3656 ").df())

In [ ]:
dfp = sme.pivot_table('imprese', columns = 'concedente', index = 'anno', fill_value=0, aggfunc = 'count').astype(int)
dfp1 = sme.pivot_table('imprese', columns = 'regione', index = 'concedente', fill_value=0, aggfunc = 'count',margins = True, margins_name = 'Total').astype(int)# 
dfp1

In [ ]:
pd.options.display.float_format = '{:.0f}'.format
sme.describe(include='all')

In [ ]:
sme.groupby(['concedente','regione'])['imprese'].count()

In [ ]:
sme.groupby(['regione','concedente']).count().unstack().plot(kind='bar',y='NOMINALE', stacked=True)

In [ ]:
# plot the dataframe
ax = dfp.plot(kind='barh', stacked=True, rot=1, figsize=(15, 7))

# add labels
for c in ax.containers:
    ax.bar_label(c, label_type='center')
_ = ax.bar_label(ax.containers[-1])

In [ ]:
sme = (con.execute("SELECT * FROM '*.parquet' where CAR = 3656").df())

In [ ]:
sme.sort_values(by='concessione', ascending=False).head(20)

# 24682 Supportitalia

In [ ]:
df.describe(include='all')

In [ ]:
Supportitalia = con.execute("select  * from '*.parquet' where car in (24682) \
and year(data_concessione) =2023 and month(data_concessione) > 0 order by 1;").df() # 

# 19795 AdE - Credito d'imposta per investimenti nel Mezzogiorno

- Credito d'imposta per investimenti nel Mezzogiorno, art.1,c.98-108, l. 208/2015, modif.art. 7-quater DL 243/2016, art.1 c.319, l. 160/2019 e art. 1 c. 171 l.178/2020, e nelle ZES art. 5,c. 2, DL 91/2017, conv. l. 123/2017, mod. art 1 c. 316 l. 160/2019
- SA.61598
- Legge 28 dicembre 2015, n. 208 (legge di Stabilità 2016)
- agenzia delle entrate
- https://www.gazzettaufficiale.it/eli/gu/2020/12/30/322/so/46/sg/pdf;  
- https://www.agenziaentrate.gov.it/portale/web/guest/schede/agevolazioni/credito-di-imposta-per-gli-investimenti-nel-mezzogiorno/scheda-informativa-investimenti-mezzogiorno-2016


In [ ]:
(con.execute("SELECT * FROM '*.parquet' where CAR = 19795").df())